In [3]:
pip install s3fs

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: C:\Users\Sicario\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
from kafka import KafkaConsumer
from time import sleep
from json import dumps,loads
import json
from s3fs import S3FileSystem
from datetime import datetime, timedelta

In [ ]:
def process_message(message):
    """Process each Kafka message and extract date"""
    try:
        data = message.value
        if isinstance(data, dict):
            # For single record messages
            flight_date = data.get('Date')
            if flight_date:
                return datetime.strptime(flight_date, '%Y-%m-%d').strftime('%Y-%m-%d')
        elif isinstance(data, list):
            # For batch messages
            first_record = data[0]
            return first_record.get('Date')
    except Exception as e:
        print(f"Error processing message: {str(e)}")
    return datetime.now().strftime('%Y-%m-%d')  # Fallback to current date

In [ ]:
consumer = KafkaConsumer(
    'demo_test',
     bootstrap_servers=[':9092'], #add your IP here ec2 public
    value_deserializer=lambda x: loads(x.decode('utf-8')))

In [ ]:
# for c in consumer:
#     print(c.value)

In [ ]:
s3 = S3FileSystem()

In [ ]:
for message in consumer:
    try:
        # Determine date from message
        date_str = process_message(message)
        
        # Create S3 path with date folder
        s3_path = f"s3://{BUCKET_NAME}/flight_data/{date_str}/flights_{message.offset}.json"
        
        # Write to S3
        with s3.open(s3_path, 'w') as file:
            json.dump(message.value, file)
        
        print(f"Saved to {s3_path}")
        
    except Exception as e:
        print(f"Error saving to S3: {str(e)}")   